## Primary tests and ideas

In [ ]:
from google.colab import drive
drive.mount('/content/drive/') 

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
from google.colab import drive
drive.mount('/content/drive/') 

import pandas as pd
import ast 
from pprint import pprint
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

data = pd.read_csv('/content/drive/My Drive/tunisia.csv')

color_list=['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'white', 'pink', 'lightblue', 'lightgreen', 'gray', 'black', 'lightgray','darkblue']

def remove_features(data):
    df = data.copy()
    to_remove = ['level','quadkey','bounding_box','name_local','name_local',
                 'name_en','name_translated','url','marker','class','parents'
                 ,'customer_rating','hotel_star_rating',
                 'hotel_star_rating_unofficial','thumbnail_url','rating',
                 'rating_local','duration_estimate','perex']
    return df.drop(to_remove,axis = 1)

def clean_location(data):
    df3 = data.copy()
    lat = []
    lng = []
    for index, row in df3.iterrows():
      d = ast.literal_eval(row['location'])
      lat.append(d['lat'])
      lng.append(d['lng'])

    #print(len(lat),len(lng))
    df3['lat'] = lat
    df3['lng'] = lng
    df3.drop('location',axis = 1, inplace=True)
    return df3



def find_Tunis_pois(df):
 
  regions = ['Ben Arous', 'Aryanah', 'Carthage', 'Twns', 'Tunis', 'Manouba', 
           'Medina of Tunis', 'La Goulette', 'Hy alkhdra Az Zuhoor']
  res = []
  geo = df.copy()
  for index,row in geo.iterrows():
    current = row['name_suffix'].split(',')
    yes = False
    for r in regions:
      if r in current:
        yes = True
        break
    
    if yes:
      name = row['name']
      suffix = row['name_suffix']
      lat = row['lat']
      lng = row['lng']
      index = row['id']
      tags = row['categories'] +', '+row['tag_keys']
      res.append({'id':index,'name':name,'suffix':suffix,'lat':lat,'lng':lng,'tags':tags})
  vis = pd.DataFrame(res)
  return vis



data = remove_features(data)
data = clean_location(data)
df = find_Tunis_pois(data)
df.head()

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


,id,name,suffix,lat,lng,tags
0,poi:12785108,UBCI,"Ben Arous, Tunisia",36.747554,10.309301,"[], ['Bank']"
1,poi:24585120,BIAT (Banque Internationale arabe de Tunisie),"Ben Arous, Tunisia",36.748062,10.309421,"[], ['Bank']"
2,poi:12781501,Bnk alaskan,"Ben Arous, Tunisia",36.746653,10.308286,"[], ['Bank']"
3,poi:12781484,ATB (Arab Tunisian Bank),"Ben Arous, Tunisia",36.741143,10.301130,"[], ['Bank']"
4,poi:35965864,Albnk altwnsy lltdamn,"Tunis, Tunisia",36.816811,10.183275,"[], ['Bank']"


In [ ]:
# regions = set()
# for index, row in data.iterrows():
#   regions.add(row['name_suffix'])
# regions

In [ ]:
df.shape

(6103, 6)

In [ ]:
from sklearn.model_selection import train_test_split

x1,x2,y1,y2 = train_test_split(df,[0]*df.shape[0],test_size=0.2)

In [ ]:
x2.shape

(1221, 6)

In [ ]:
import folium

def show_pois(data):
  vis = data.copy()  
  locations = vis[['lat', 'lng']]
  
  popup = vis['name']
  locationlist = locations.values.tolist()
  
  color_list=['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'white', 'pink', 'lightblue', 'lightgreen', 'gray', 'black', 'lightgray','darkblue']
  map = folium.Map(location=[36.853685 , 10.323380 ], zoom_start=12,tiles='Stamen Terrain')

  for point in range(0,len(locationlist)):
      folium.Marker(locationlist[point], icon=folium.Icon(color='blue'),popup=vis.iloc[point,1]).add_to(map)

  return map

#show_folium(x2)

In [ ]:
import random
random.seed(42)

In [ ]:
ind = random.randint(0,df.shape[0])
poi = df.loc[ind]
def show_one_poi(poi):
  locations = poi[['lat', 'lng']]
  
  pop = poi['name']
  locationlist = locations.values.tolist()
  
  map = folium.Map(location=[poi['lat'] , poi['lng'] ], zoom_start=12,tiles='Stamen Terrain')

  
  folium.Marker(locationlist, icon=folium.Icon(color='green'),popup=pop).add_to(map)

  return map
#show_one_poi(poi)

#### Experimentation

In [ ]:
random.sample(range(df.shape[0]), 10)

[912, 204, 6074, 2253, 2006, 1828, 1143, 6033, 839, 5543]

In [ ]:
import random
random.seed(42)
#first select a random sample of POIs
def select_home_pois(nb,df):
  return random.sample(range(df.shape[0]), nb)

def select_random_pois(nb,df):
  lst = []
  inds = random.sample(range(df.shape[0]), nb)
  return df.loc[inds]

nb = 100
steps = [3,10,20]

homes = select_random_pois(nb,df)
homes.head()

,id,name,suffix,lat,lng,tags
5238,poi:35289578,Tunis Marine,"Twns, Tunisia",36.800733,10.192072,"['traveling'], ['Stop', 'Public Transport']"
912,poi:6475807,Annawress,"Ben Arous, Tunisia",36.709908,10.206283,"['eating'], ['Café']"
204,poi:36034699,BIAT,"Twns, Tunisia",36.805689,10.177554,"[], ['Bank']"
6074,poi:21787183,Parking,"Manouba, Tunisia",36.815145,10.102958,"['traveling'], ['Parking']"
2253,poi:12641653,Mon Boucher,"Twns, Tunisia",36.857672,10.266689,"['shopping'], ['Butcher', 'Shop']"


In [ ]:
!pip install haversine

In [ ]:
#We need then to be able to calculate distance between pois
#This should be the basic idea for the HOME POI
poi1 = homes.loc[5238]
poi2 = homes.loc[6074]
def haversine_distance(lat1, lon1, lat2, lon2):  
   r = 6371  
   phi1 = np.radians(lat1)  
   phi2 = np.radians(lat2)  
   delta_phi = np.radians(lat2 - lat1)  
   delta_lambda = np.radians(lon2 - lon1)  
   a = np.sin(delta_phi / 2)**2 + np.cos(phi1) * np.cos(phi2) *   np.sin(delta_lambda / 2)**2  
   res = r * (2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a)))  
   return np.round(res, 2)

def geo_distance(poi1,poi2):
  return haversine_distance(poi1['lat'],poi1['lng'],poi2['lat'],poi2['lng'])

geo_distance(poi1,poi2)

8.09

In [ ]:
import haversine as hs
hs.haversine(poi1[['lat','lng']],poi2[['lat','lng']])

8.09393406845978

In [ ]:
show_one_poi(poi1)

In [ ]:
show_one_poi(poi2)

In [ ]:
ind = homes.index[0]
ind

5238

In [ ]:
#Now we cluster/ or select the Work POIs according to a distance threshold (step)
radius, margin = 10, 0.5
def cluster_by_radius(df,radius,margin,ind):
  clustered = []
  poi1 = df.loc[ind]
  for index, row in df.iterrows():
    if index != ind:
      poi2 = df.loc[index]
      dist = geo_distance(poi1,poi2)
      if abs(dist - radius) <= margin:
        clustered.append(poi2)
  return pd.DataFrame(clustered)

      
clustered1 = cluster_by_radius(df,radius,margin,ind)
clustered1.head()

,id,name,suffix,lat,lng,tags
22,poi:24454223,Altjary bnk,"Aryanah, Tunisia",36.889495,10.181650,"[], ['Bank']"
38,poi:36065228,WIFAK BANK,"Twns, Tunisia",36.835596,10.297928,"[], ['Bank']"
53,poi:16658677,Amen Bank,"Tunis, Tunisia",36.845913,10.284612,"[], ['Bank']"
65,poi:11966928,UIB,"Twns, Tunisia",36.815428,10.304123,"[], ['Bank']"
67,poi:21822710,BIAT,"Aryanah, Tunisia",36.862962,10.276385,"[], ['Bank']"


In [ ]:
poi = df.loc[ind]
ind

5238

In [ ]:
def show_poi_clustered(clustered,df,poi):
 
  locations = clustered[['lat', 'lng']]
  
  locationlist = locations.values.tolist()
  
  color_list=['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'white', 'pink', 'lightblue', 'lightgreen', 'gray', 'black', 'lightgray','darkblue']
  map = folium.Map(location=[poi['lat'] , poi['lng'] ], zoom_start=12,tiles='Stamen Terrain')

  folium.Marker(location=[poi['lat'] , poi['lng'] ],icon=folium.Icon(color='green'),popup=df.iloc[ind,1]).add_to(map)

  for point in range(0,len(locationlist)):
      folium.Marker(locationlist[point], icon=folium.Icon(color='blue'),popup=df.iloc[point,1]).add_to(map)

  return map


show_poi_clustered(clustered1,df,poi)


In [ ]:
random.choices(list((clustered1.index)))

[715]

In [ ]:
def select_one_work(clustered):
  return random.choices(list((clustered.index)))

ind1 = select_one_work(clustered1)
ind1 = ind1[0]

In [ ]:
ind,ind1

(5238, 4902)

In [ ]:
def show_home_work(home,work,df):

  home_loc = df.loc[home][['lat','lng']]
  work_loc = df.loc[work][['lat','lng']]
  
  map = folium.Map(location=[home_loc['lat'] , home_loc['lng'] ], zoom_start=12,tiles='Stamen Terrain')

  folium.Marker(location=[home_loc['lat'] , home_loc['lng'] ],
                icon=folium.Icon(color='green'),
                popup=df.iloc[home,1]).add_to(map)

  folium.Marker(location=[work_loc['lat'] , work_loc['lng'] ],
                icon=folium.Icon(color='red'),
                popup=df.iloc[work,1]).add_to(map)


  return map

show_home_work(ind,ind1,df)

In [ ]:
random.seed(42)
radius, margin = 3, 0.1
home = 1905
clustered1 = cluster_by_radius(df,radius,margin,home)
clustered1.head() #I see, so we have to select good Home pois too, and not just randomly
#in other words, change the algorithm, make it one step execution for every row
#select one home randomly -> find if it has nearby neighboors -> if yes pick random work
#else reject it and find another one... it should converge

""


In [ ]:
random.seed(42)
def associate_home_work(df,number,step):
  radius, margin = step, 0.1
  result = []
  try:
    homes = select_random_pois(number,df)
    homes = list(homes.index)
    for home in homes:
      try:
        clustered1 = cluster_by_radius(df,radius,margin,home)
        print('clustered')
        work = select_one_work(clustered1)[0]
        print('selected')
        result.append({'home':home,'work':work})
        print('appended')
      except Exception as e:
        print(home,clustered1)
        print('Error shiimatta ',e)
        return pd.DataFrame()
    print('Success')
    return pd.DataFrame(result)

  except Exception as e:
    print('Error ',e)
    return


associated = associate_home_work(df,100,3) #40% with step 3km
associated.head()

clustered
selected
appended
clustered
selected
appended
clustered
selected
appended
clustered
selected
appended
clustered
selected
appended
clustered
selected
appended
clustered
selected
appended
clustered
selected
appended
clustered
selected
appended
clustered
selected
appended
clustered
selected
appended
clustered
selected
appended
clustered
selected
appended
clustered
selected
appended
clustered
selected
appended
clustered
selected
appended
clustered
selected
appended
clustered
selected
appended
clustered
selected
appended
clustered
selected
appended
clustered
1905 Empty DataFrame
Columns: []
Index: []
Error shiimatta  list index out of range


""


In [ ]:
err1 = df.loc[1905]
show_one_poi(err1)

## Resulting algorithm

In [ ]:
import random

def select_home_pois(nb,df):
  return random.sample(range(df.shape[0]), nb)

def select_random_pois(nb,df):
  lst = []
  inds = random.sample(range(df.shape[0]), nb)
  return df.loc[inds]

def haversine_distance(lat1, lon1, lat2, lon2):  
   r = 6371  
   phi1 = np.radians(lat1)  
   phi2 = np.radians(lat2)  
   delta_phi = np.radians(lat2 - lat1)  
   delta_lambda = np.radians(lon2 - lon1)  
   a = np.sin(delta_phi / 2)**2 + np.cos(phi1) * np.cos(phi2) *   np.sin(delta_lambda / 2)**2  
   res = r * (2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a)))  
   return np.round(res, 2)

def geo_distance(poi1,poi2):
  return haversine_distance(poi1['lat'],poi1['lng'],poi2['lat'],poi2['lng'])

def cluster_by_radius(df,radius,margin,ind):
  clustered = []
  poi1 = df.loc[ind]
  for index, row in df.iterrows():
    if index != ind:
      poi2 = df.loc[index]
      dist = geo_distance(poi1,poi2)
      if abs(dist - radius) <= margin:
        clustered.append(poi2)
  return pd.DataFrame(clustered)

def select_one_work(clustered):
  return random.choices(list((clustered.index)))

In [ ]:

random.seed(42)
#this is the resulting algorithm
#first it selects randomly a home from the dataset
#then it finds the neighboors or the corresponding 'cluster' according to the 
#step or range
#pick one of the cluster randomly and assign it as the work 
#repeat the process to reach the total required number for the sample

def associate_home_work_better(df,number,step):
  radius, margin = step, 0.1
  result = []
  try:
    indexes = list(df.index)
    try:
      while number>0:
        home = random.choice(indexes)
        clustered1 = cluster_by_radius(df,radius,margin,home)
        if clustered1.shape[0] > 0:
          
          work = select_one_work(clustered1)[0]
          result.append({'home':home,'work':work})
          number-=1
        else:
          print('here!',number)
          continue
        
    except Exception as e:
      print('Error shiimatta ',e)
      return None
      
    print('Success')
    return pd.DataFrame(result)

  except Exception as e:
    print('Error ',e)
    return


associated = associate_home_work_better(df,100,10) 
associated.head()

Success


,home,work
0,5238,843
1,6074,1466
2,1828,382
3,839,5713
4,4467,524


In [ ]:
a = associated.loc[associated.index[1]]
show_home_work(a['home'],a['work'],df)

In [ ]:
def show_result(result,df):
  
  map = folium.Map(location=[36.853685 , 10.323380 ], zoom_start=20,tiles='Stamen Terrain')

  for index,row in result.iterrows():
    home_loc = df.loc[row['home']][['lat','lng']]
    work_loc = df.loc[row['work']][['lat','lng']]
    
    

    folium.Marker(location=[home_loc['lat'] , home_loc['lng'] ],
                  icon=folium.Icon(color='green'),
                  popup=df.iloc[row['home'],1]).add_to(map)

    folium.Marker(location=[work_loc['lat'] , work_loc['lng'] ],
                  icon=folium.Icon(color='red'),
                  popup=df.iloc[row['work'],1]).add_to(map)


  return map

show_result(associated,df)

#### An advanced way to generate more POIs for HOME


In [ ]:
import math
def generateRandomPoints(center, radius, count):
  points = []
  for i in range(count):
    points.append(generateRandomPoint(center,radius))
  return points

def generateRandomPoint(center,radius):
  x0 = center['lng'] #lng
  y0 = center['lat'] #lat
  rd = radius / 111300
  u = random.random()
  v = random.random()

  w = rd * math.sqrt(u)
  t = 2 * np.pi * v
  x = w * np.cos(t)
  y = w * np.sin(t)

  xp = x / np.cos(y0)

  return {'lat': y+y0, 'lng': xp+x0}

source = df.loc[associated.index[0]]
randomGeoPoints = generateRandomPoints(source, 100, 10)
randomGeoPoints

[{'lat': 36.747583238104745, 'lng': 10.30895824813418},
 {'lat': 36.747386587807185, 'lng': 10.30784521703073},
 {'lat': 36.74754346423336, 'lng': 10.30893246135855},
 {'lat': 36.74789998887043, 'lng': 10.310599155060595},
 {'lat': 36.747375244426934, 'lng': 10.309561968624859},
 {'lat': 36.747349164552624, 'lng': 10.310219350469795},
 {'lat': 36.74723134771687, 'lng': 10.308273551794853},
 {'lat': 36.7476422377715, 'lng': 10.30956975031947},
 {'lat': 36.74726373730741, 'lng': 10.30815005472838},
 {'lat': 36.74805820604606, 'lng': 10.309242348936387}]

In [ ]:
random.choice(randomGeoPoints)

{'lat': 36.747349164552624, 'lng': 10.310219350469795}

In [ ]:
lst = []
for c in randomGeoPoints:
  lst.append([c['lat'],c['lng']])

def show_folium(lst):
  
  locationlist = lst[:]
  map = folium.Map(location=[36.853685 , 10.323380 ], zoom_start=12,tiles='Stamen Terrain')

  for point in range(0,len(locationlist)):
    print(locationlist[point])
    
    folium.Marker(locationlist[point], icon=folium.Icon(color='green'),popup=str(point)).add_to(map)

  return map

show_folium(lst)

[36.747583238104745, 10.30895824813418]
[36.747386587807185, 10.30784521703073]
[36.74754346423336, 10.30893246135855]
[36.74789998887043, 10.310599155060595]
[36.747375244426934, 10.309561968624859]
[36.747349164552624, 10.310219350469795]
[36.74723134771687, 10.308273551794853]
[36.7476422377715, 10.30956975031947]
[36.74726373730741, 10.30815005472838]
[36.74805820604606, 10.309242348936387]
